<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Find and Delete ECS (Elastic Container Service) Clusters with Low CPU Utilization</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-ECS-Clusters-with-Low-CPU-Utilization"><u>Delete ECS Clusters with Low CPU Utilization</u><a class="jp-InternalAnchorLink" href="#Delete-ECS-Clusters-with-Low-CPU-Utilization" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find ECS Clusters with Low CPU Utilization</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete ECS Clusters</a></p>

In [ ]:
if region == None:
    region = ''
if cluster_names and not region:
    raise SystemExit("Provide a region for the ECS Cluster names!")

<h3 id="Find-ECS-Clusters-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find ECS Clusters with Low CPU Utilization</h3>
<p>Using unSkript's Find ECS Clusters with Low CPU Utilization action, we will find clusters with a low CPU utilization given a threshold percentage using the&nbsp;<span style="color: rgb(53, 152, 219);">CPUUtilization <span style="color: rgb(0, 0, 0);">attribue found in the statistics list of&nbsp; <em>descibe_clusters</em> API.</span></span></p>
<blockquote>
<p>This action takes the following parameters: <code>region, threshold</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>low_cpu_clusters</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions


from beartype import beartype
@beartype
def aws_list_clusters_with_low_utilization_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_list_clusters_with_low_utilization(handle, region: str = "", threshold: int = 10) -> Tuple:
    """aws_list_clusters_with_low_utilization Returns an array of ecs clusters.

        :type region: string
        :param region: AWS Region.

        :type threshold: int
        :param threshold: (In percent) Threshold to check for cpu utilization
        is less than threshold.

        :rtype: List of clusters for low CPU utilization
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            ecs_Client = handle.client('ecs', region_name=reg)
            response = aws_get_paginator(ecs_Client, "list_clusters", "clusterArns")
            for cluster in response:
                cluster_dict = {}
                cluster_name = cluster.split('/')[1]
                stats = ecs_Client.describe_clusters(clusters=[cluster])['clusters'][0]['statistics']
                for stat in stats:
                    if stat['name'] == 'CPUUtilization':
                        cpu_utilization = int(stat['value'])
                        if cpu_utilization < threshold:
                            cluster_dict["cluster_name"] = cluster_name
                            cluster_dict["region"] = reg
                            result.append(cluster_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "threshold": "int(threshold)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not cluster_names",
    "condition_result": true
    }''')

task.configure(outputName="low_cpu_clusters")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_clusters_with_low_utilization, lego_printer=aws_list_clusters_with_low_utilization_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Low-CPU-Utilization-Clusters">Create List of Low CPU Utilization Clusters</h3>
<p>This action filters regions that have no clusters with low CPU utilization and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_low_cpu_clusters</code></p>
</blockquote>

In [ ]:
all_low_cpu_clusters = []
try:
    if low_cpu_clusters[0] == False:
        if len(low_cpu_clusters[1])!=0:
            all_low_cpu_clusters=low_cpu_clusters[1]
except Exception:
    for vol_id in volume_ids:
        data_dict = {}
        data_dict["region"] = region
        data_dict["volume_id"] = vol_id
        all_low_cpu_clusters.append(data_dict)
print(all_low_cpu_clusters)
task.configure(outputName="all_low_cpu_clusters")

<h3 id="Delete-Low-Usage-EBS-Volumes"><a id="2" target="_self" rel="nofollow"></a>Delete ECS Cluster</h3>
<p>This action deletes clusters found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>volume_id, region</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_ecs_cluster_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_ecs_cluster(handle, region: str, cluster_name: str) -> Dict:
    """aws_delete_ecs_cluster dict of loadbalancers info.

        :type region: string
        :param region: AWS Region.

        :type cluster_name: string
        :param cluster_name: ECS Cluster name

        :rtype: dict of load balancers info.
    """
    try:
        ec2Client = handle.client('ecs', region_name=region)
        response = ec2Client.delete_cluster(cluster=cluster_name)
        return response
    except Exception as e:
        raise Exception(e)




task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "cluster_name": "iter.get(\\"cluster_name\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_low_cpu_clusters",
    "iter_parameter": ["region","cluster_name"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_low_cpu_clusters)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_ecs_cluster, lego_printer=aws_delete_ecs_cluster_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to filter low CPU utilization ECS clusters given threshold percentage and delete them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>